Here we will show an example of how to run deepblast on simulation data.

In [1]:
import os
from deepblast.sim import hmm_alignments
import argparse
import numpy as np

We will first simulate multiple sequences from a single PFam family.
The resulting generated alignments will be used to train the model.

In [2]:
hmm = '../data/zf-C2H2.hmm'
n_alignments = 100
np.random.seed(0)
align_df = hmm_alignments(n=40, seed=0, n_alignments=n_alignments, hmmfile=hmm)

cols = [
    'chain1_name', 'chain2_name', 'tmscore1', 'tmscore2', 'rmsd',
    'chain1', 'chain2', 'alignment'
]
align_df.columns = cols

The simulated alignments will be split into training / testing and validation.

In [3]:
parts = n_alignments // 10
train_df = align_df.iloc[:parts * 8]
test_df = align_df.iloc[parts * 8:parts * 9]
valid_df = align_df.iloc[parts * 9:]

# save the files to disk.
train_df.to_csv('../data/train.txt', sep='\t', index=None, header=None)
test_df.to_csv('../data/test.txt', sep='\t', index=None, header=None)
valid_df.to_csv('../data/valid.txt', sep='\t', index=None, header=None)

We will prepare the environment to make sure that the appropriate output directories exist to store the results.

In [4]:
from deepblast.trainer import DeepBLAST
from pytorch_lightning import Trainer

output_dir = 'simulation_results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [5]:
os.getcwd()

'/home/juermieboop/Documents/software/deepblast/ipynb'

We will now create the arguments.  Below is the way to create this in a python environment.
This can also be recreated on a standard command line interface.

In [6]:
args = [
    '--train-pairs', f'{os.getcwd()}/../data/train.txt',
    '--test-pairs', f'{os.getcwd()}/../data/test.txt',
    '--valid-pairs', f'{os.getcwd()}/../data/valid.txt',
    '--output-directory', output_dir,
    '--epochs', '32',
    '--batch-size', '20',   
    '--num-workers', '1',
    '--learning-rate', '1e-4',
    '--layers', '1',
    '--visualization-fraction', '1',
    '--loss', 'cross_entropy',
    '--scheduler', 'cosine',
    '--gpus', '1'
]
parser = argparse.ArgumentParser(add_help=False)
parser = DeepBLAST.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)

In [7]:
%tb

No traceback available to show.


We will then initialize the alignment model with the parameters we specified earlier.

In [8]:
model = DeepBLAST(args)

We can now train the model.

In [9]:
trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=1,
    # profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)

trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | aligner | NeedlemanWunschAligner | 820 K 
---------------------------------------------------
820 K     Trainable params
0         Non-trainable params
820 K     Total params
/home/juermieboop/miniconda3/envs/deepblast/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

predA torch.Size([10, 32, 32]) theta torch.Size([10, 32, 32]) gap torch.Size([10, 32, 32])
x_str FPCPHCNRPMKFVHQPNFSRHVFIDH 26
y_str YTCSYCSLSFANSQALMKHALIH 23
x_str FKCASQDCDTSFVTQANRTYHLCRH 25
y_str YTCSYCSLSFANSQALMKHALIH 23
x_str YVCPTYCGYSSKSSIGVTAHLREGH 25
y_str FKCHDCERGFAQPDMLESHVQKH 23
x_str FKCHDCERGFAQPDMLESHVQKH 23
y_str YQCDMICYHCIVTRFSQSDTFARHSKYPH 29
x_str FSCGRDCHRKFSTNYYLTGHLTTH 24
y_str FSCGRDCHRKFSTNYYLTGHLTTH 24
x_str LKCSRYCDPHFPAQDDLNQHRKHH 24
y_str FNCGGCTEDICFSSMFNLHMSIH 23
x_str FRCNVCDKPFIDPISLNEHNSTH 23
y_str FLCMNGCSKRFKKDFLLNAHKRRH 24
x_str YTCPDCFLAFSDPAHLTNHMSSLH 24
y_str TNCSACGMPFEQPKQFTRHRRNSH 24
x_str LPCKVNRCLPRCSEGCNNTTKFTRHMRRMH 30
y_str YQCDMICYHCIVTRFSQSDTFARHSKYPH 29
x_str FNCGGCTEDICFSSMFNLHMSIH 23
y_str YICPKCEKVTTHASHLTRHVRLH 23


/home/juermieboop/miniconda3/envs/deepblast/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/home/juermieboop/miniconda3/envs/deepblast/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/home/juermieboop/miniconda3/envs/deepblast/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottlene

Training: 0it [00:00, ?it/s]

AttributeError: 'numpy.ndarray' object has no attribute 'to'

The model diagnostics can be directly visualized in Tensorboard. Here, we show the losses, the accuracy and the alignment results.

In [ ]:
!ls lightning_logs

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs

And we did this with just a few million parameters

In [ ]:
model.aligner

In [ ]:
!ls lightning_logs/version_29/checkpoints

In [ ]:
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
checkpoint_dir = 'lightning_logs/version_29/checkpoints'
path = f'{checkpoint_dir}/epoch=25.ckpt'
model = LightningAligner.load_from_checkpoint(path)

In [ ]:
from deepblast.dataset.dataset import states2matrix, tmstate_f
import numpy as np
import torch
from torch.nn.utils.rnn import pack_sequence
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
i = 9
test = align_df
x = str.encode(test.iloc[i]['chain1'])
y = str.encode(test.iloc[i]['chain2'])
states = test.iloc[i]['alignment']
x_ = torch.Tensor(model.tokenizer(x)).long().unsqueeze(0)
y_ = torch.Tensor(model.tokenizer(y)).long().unsqueeze(0)
x_ = pack_sequence(x_, enforce_sorted=False).cuda()
y_ = pack_sequence(y_, enforce_sorted=False).cuda()
states = list(map(tmstate_f, states))
A = states2matrix(states)

In [ ]:
model = model.cuda()
aln, theta, gap = model.forward(x_, y_)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(16, 3))
sns.heatmap(A, ax=ax[0])
sns.heatmap(aln.cpu().detach().numpy().squeeze(), ax=ax[1])
sns.heatmap(np.log(theta.cpu().detach().numpy().squeeze()), ax=ax[2])
sns.heatmap(gap.cpu().detach().numpy().squeeze(), ax=ax[3])

In [ ]:
from deepblast.dataset.alphabet import Uniprot21
from deepblast.dataset.dataset import decode
from deepblast.score import alignment_text, roc_edges

In [ ]:
from deepblast.dataset.dataset import states2edges

#pred_edges = list(zip(pred_y, pred_x))
pred_edges = states2edges(pred_states)
true_edges = states2edges(truth_states)
stats = roc_edges(true_edges, pred_edges)

In [ ]:
gen = model.aligner.traceback(x_, y_)
decoded, _ = next(gen)
pred_x, pred_y, pred_states = list(zip(*decoded))
pred_states = np.array(list(pred_states))
truth_states = np.array(list(states))                    
text = alignment_text(x.decode('utf-8'), y.decode('utf-8'), pred_states, truth_states, stats)

In [ ]:
px, py = zip(*pred_edges)
tx, ty = zip(*true_edges)

plt.plot(px, py, label='pred')
plt.plot(tx, ty, label='truth')

In [ ]:
from deepblast.dataset.dataset import state_diff_f
def states2edges(states):
    """ Converts state string to bipartite matching. """
    prev_s, next_s = states[:-1], states[1:]
    transitions = list(zip(prev_s, next_s))
    state_diffs = np.array(list(map(state_diff_f, transitions)))
    coords = np.cumsum(state_diffs, axis=0).tolist()
    coords = [(0, 0)] + list(map(tuple, coords))
    return coords
pe = states2edges(list(pred_states))
te = states2edges(list(truth_states))

In [ ]:
px, py = zip(*pe)
tx, ty = zip(*te)

plt.plot(px, py, label='pred')
plt.plot(tx, ty, label='truth')

In [ ]:
pe

In [ ]:
te

In [ ]:
truth_states

In [ ]:
pred_states